In [1]:
import sys
import os
import json
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt

# Add project root to path
sys.path.append(os.path.abspath('..'))

from src.data.loader import NuScenesLoader

# Initialize Data Loader
print("⏳ Loading NuScenes Database...")
loader = NuScenesLoader()
print("✅ Ready.")

⏳ Loading NuScenes Database...
Loading NuScenes v1.0-mini database from /mnt/d/Nuscenes_dataset...
✅ Ready.


In [2]:
def create_front_panorama(paths, resize_h=300):
    """
    Stitches Front-Left, Front, Front-Right into a single panoramic strip.
    """
    order = ["CAM_FRONT_LEFT", "CAM_FRONT", "CAM_FRONT_RIGHT"]
    images = []
    
    # 1. Load Images
    for cam in order:
        if cam in paths:
            try:
                img = Image.open(paths[cam])
                # Resize keeping aspect ratio
                aspect = img.width / img.height
                new_w = int(resize_h * aspect)
                img = img.resize((new_w, resize_h), Image.Resampling.LANCZOS)
                images.append((cam, img))
            except: pass
            
    if not images: return None

    # 2. Create Canvas
    total_w = sum(img.width for _, img in images)
    max_h = max(img.height for _, img in images)
    
    pano = Image.new('RGB', (total_w, max_h))
    draw = ImageDraw.Draw(pano)
    
    # 3. Paste & Label
    x_offset = 0
    for cam_name, img in images:
        pano.paste(img, (x_offset, 0))
        
        # Add Label
        label = cam_name.replace("CAM_", "")
        draw.rectangle([x_offset+5, 5, x_offset+100, 25], fill="black")
        draw.text((x_offset+10, 10), label, fill="white")
        
        x_offset += img.width
        
    return pano

def load_results(filename):
    """Parses the JSONL output file into a list."""
    data = []
    filepath = os.path.join("..", "output", filename)
    
    if not os.path.exists(filepath):
        print(f"File not found: {filepath}")
        return []
        
    with open(filepath, 'r') as f:
        for line in f:
            try:
                item = json.loads(line)
                # Filter out errors
                if item.get("error") or not item.get("token"): continue
                data.append(item)
            except: pass
    return data

def generate_label(item, index):
    """Smart label for dropdown."""
    # 1. WOD-E2E Tags
    wod = item.get("wod_e2e_tags", [])
    
    # 2. Weather
    env = item.get("environment", {})
    weather = env.get("weather", "")
    
    # 3. Criticality
    beh = item.get("behavioral_dynamics", {})
    score = beh.get("criticality_score", item.get("criticality_score", 0))

    tags = []
    if wod: tags.append(f"🚩 {','.join(wod)}")
    if weather and weather != "clear": tags.append(f"☁️ {weather}")
    if int(score or 0) >= 7: tags.append(f"⚠️ Lv{score}")
    
    tag_str = " | ".join(tags) if tags else "Nominal"
    return f"{index}. [{tag_str}]"

In [3]:
# --- WIDGETS ---

# 1. File Selection (Scans output folder)
output_dir = os.path.join("..", "output")
files = [f for f in os.listdir(output_dir) if f.startswith('index_') and f.endswith('.jsonl')]
file_dropdown = widgets.Dropdown(
    options=files,
    description='Result File:',
    layout=widgets.Layout(width='50%')
)

# 2. Frame Selection
frame_dropdown = widgets.Dropdown(
    description='Scenario:',
    layout=widgets.Layout(width='90%')
)

# 3. Output Areas
out_visual = widgets.Output()
out_analysis = widgets.Output()

# --- STATE ---
current_data = []

def on_file_change(change):
    global current_data
    if change['type'] == 'change' and change['name'] == 'value':
        filename = change['new']
        current_data = load_results(filename)
        
        # Populate Dropdown
        options = []
        for i, item in enumerate(current_data):
            label = generate_label(item, i)
            options.append((label, i))
        
        frame_dropdown.options = options
        frame_dropdown.value = 0 if options else None

def on_frame_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        idx = change['new']
        if idx is None: return
        
        item = current_data[idx]
        token = item['token']
        
        # A. Render Visuals
        with out_visual:
            clear_output(wait=True)
            try:
                paths = loader.get_camera_paths(token)
                pano = create_front_panorama(paths)
                if pano:
                    display(pano)
                else:
                    print("No images found for this token.")
            except Exception as e:
                print(f"Visual Error: {e}")

        # B. Render Analysis
        with out_analysis:
            clear_output(wait=True)
            
            # 1. The Brain (Reasoning Trace)
            trace = item.get('_reasoning_trace', 'No trace found.')
            print("🧠 REASONING TRACE:")
            print("-" * 80)
            print(trace)
            print("-" * 80 + "\n")
            
            # 2. The Output (JSON)
            display_item = item.copy()
            # Remove bulky fields for display
            display_item.pop('token', None)
            display_item.pop('_reasoning_trace', None)
            display_item.pop('model_source', None)
            
            print("📋 STRUCTURED OUTPUT:")
            print(json.dumps(display_item, indent=2))

# Bind
file_dropdown.observe(on_file_change)
frame_dropdown.observe(on_frame_change)

# Init
if files:
    on_file_change({'type': 'change', 'name': 'value', 'new': files[0]})

# Layout
ui = widgets.VBox([
    file_dropdown,
    frame_dropdown,
    out_visual,
    out_analysis
])

display(ui)